In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI

In [2]:
print('Enter mail address')
email = input()

print('Enter password')
password = getpass()

server_url = r'https://api.quantum-inspire.com'
authentication = BasicAuthentication(email, password)
qi = QuantumInspireAPI(server_url, authentication)

Enter mail address
Y.Zhang-108@student.tudelft.nl
Enter password
········


SSLError: HTTPSConnectionPool(host='api.quantum-inspire.com', port=443): Max retries exceeded with url: /schema/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1076)')))

In [18]:
qi.list_backend_types()

Backend type: Spin-2, number of qubits: 2
Backend type: Starmon-5, number of qubits: 5
Backend type: QX single-node simulator, number of qubits: 26
Backend type: QX single-node simulator SurfSara, number of qubits: 31


In [6]:
def parameterized_QACM_Economic(θ, backend="Starmon-5"):
    
    # here the qubits used in circuit is 1,3 instead of 0,4, for lower error rate
    
    # q[0] is Bob's qubit and q[1] is Eve‘s qubit
    
    qasm1 ='''
    version 1.0
    qubits 2
    
    # initialize the state
    Prep_z q[0]
    Prep_z q[1]
    Ry q[0], {0}
    # Prepare on XZ equator


    Rx q[0], -1.57079632675    
    Sdag q[1]
    H q[1]
    Tdag q[1]
    CNOT q[0],q[1]
    T q[1]
    H q[1]
    S q[1]
    CNOT q[1],q[0]

    #Rotate back and measure
    Rx q[0], 1.57079632675
    Rx q[1], 1.57079632675

    Ry q[1], -{0}
    Measure_z q[1]
    '''
    
    qasm2 ='''
    version 1.0
    qubits 5
    
    # initialize the state
    Prep_z q[2]
    Prep_z q[3]
    Ry q[2], {0}
    
    #
    Rx q[2], -1.57079632675
    Sdag q[3]
    H q[3]
    Tdag q[3]
    CNOT q[2],q[3]
    T q[3]
    H q[3]
    S q[3]
    CNOT q[3],q[2]

    #Rotate back and measure
    Rx q[2], 1.57079632675
    Rx q[3], 1.57079632675
    
    Ry q[3], -{0}
    Measure_z q[3]
    '''
    if backend=="Starmon-5":
        return qasm2.format(θ)
    elif backend=="Spin-2":
        return qasm1.format(θ)
    else:
        raise TypeError("Invalid Backend")

### Run on Starmon-5

In [43]:
npoints=100
N_shots=4096

backend_type = qi.get_backend_type_by_name('Starmon-5')

copied_data=[]
target_points=np.linspace(0,np.pi,npoints)

index=0
for θ in target_points:
    qasm=parameterized_QACM_Economic(θ)
    result = qi.execute_qasm(qasm, backend_type=backend_type, number_of_shots=N_shots)
    hist=result.get('histogram', {})
    copied_data.append([hist["0"],hist["8"]])# starmon-5
#     copied_data.append([hist["0"],hist["2"]])# spin-2
    index=index+1
    print(str(index)+"/"+str(npoints))
    print("Execution time: ", result.get('execution_time_in_seconds',{}))
    print("Theta: {0}".format(θ))
    print([hist["0"],hist["8"]])
#     print([hist["0"],hist["2"]])

copied_data=np.array(copied_data)

1/30
Execution time:  0.622592
Theta: 0.0
[0.946044921875, 0.053955078125]
2/30
Execution time:  0.622592
Theta: 0.10833078115826873
[0.949951171875, 0.050048828125]
3/30
Execution time:  0.622592
Theta: 0.21666156231653746
[0.948974609375, 0.051025390625]
4/30
Execution time:  0.622592
Theta: 0.3249923434748062
[0.95849609375, 0.04150390625]
5/30
Execution time:  0.622592
Theta: 0.43332312463307493
[0.954833984375, 0.045166015625]
6/30
Execution time:  0.622592
Theta: 0.5416539057913436
[0.95556640625, 0.04443359375]
7/30
Execution time:  0.622592
Theta: 0.6499846869496124
[0.943603515625, 0.056396484375]
8/30
Execution time:  0.622592
Theta: 0.7583154681078811
[0.94775390625, 0.05224609375]
9/30
Execution time:  0.622592
Theta: 0.8666462492661499
[0.926025390625, 0.073974609375]
10/30
Execution time:  0.622592
Theta: 0.9749770304244186
[0.929931640625, 0.070068359375]
11/30
Execution time:  0.622592
Theta: 1.0833078115826873
[0.92529296875, 0.07470703125]
12/30
Execution time:  0.622

In [39]:
result

OrderedDict([('id', 6864514),
             ('url', 'https://api.quantum-inspire.com/results/6864514/'),
             ('job', 'https://api.quantum-inspire.com/jobs/6867623/'),
             ('created_at', '2020-12-17T17:29:54.369096Z'),
             ('number_of_qubits', 5),
             ('execution_time_in_seconds', 0.622592),
             ('raw_text', ''),
             ('raw_data_url',
              'https://api.quantum-inspire.com/results/6864514/raw-data/06a08e1dde9b17628449d81735f057b2f56b01407096985d7e4e927c6f878072/'),
             ('histogram',
              OrderedDict([('0', 0.955810546875), ('8', 0.044189453125)])),
             ('histogram_url',
              'https://api.quantum-inspire.com/results/6864514/histogram/06a08e1dde9b17628449d81735f057b2f56b01407096985d7e4e927c6f878072/'),
             ('measurement_mask', 8),
             ('quantum_states_url',
              'https://api.quantum-inspire.com/results/6864514/quantum-states/06a08e1dde9b17628449d81735f057b2f56b0140709

### Run on Spin-2

In [ ]:
npoints=100
N_shots=4096

backend_type = qi.get_backend_type_by_name('Spin-2')

copied_data=[]
target_points=np.linspace(0,np.pi,npoints)

index=0
for θ in target_points:
    qasm=parameterized_QACM_Economic(θ)
    result = qi.execute_qasm(qasm, backend_type=backend_type, number_of_shots=N_shots)
    hist=result.get('histogram', {})
#     copied_data.append([hist["0"],hist["8"]])# starmon-5
    copied_data.append([hist["0"],hist["2"]])# spin-2
    index=index+1
    print(str(index)+"/"+str(npoints))
    print("Execution time: ", result.get('execution_time_in_seconds',{}))
    print("Theta: {0}".format(θ))
#     print([hist["0"],hist["8"]]) # Starmon-5
    print([hist["0"],hist["2"]]) # Spin-2

copied_data=np.array(copied_data)